資料處理-TOKYO（按日期跟航班代碼排序）

In [17]:
import os
import pandas as pd

def process_and_merge_files(start_date, end_date, base_path, output_path):
    # 建立日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2024-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/tokyo_{date}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if not df.empty:
                df = df.dropna(how="all")
                
                # 格式化出發日期並去除時間
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                
                # 新增星期欄位
                df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                
                # 移除價格中的符號並轉換為數字
                df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]  # 過濾掉不符合數字格式的行
                df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                
                # 計算 day left
                file_date = pd.to_datetime(f"2024-{date[:2]}-{date[2:]}")
                df['day left'] = (pd.to_datetime(df['出發日期']) - file_date).dt.days
                
                # 調整欄位順序，將「星期」移到「出發時間」後面
                cols = df.columns.tolist()
                cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('星期')))
                df = df[cols]
                
                data_frames.append(df)

    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    if merged_data.empty:
        print("沒有合併到任何資料。")
        return

    # 計算每個航班的筆數
    flight_counts = merged_data.groupby(['出發日期', '航班代碼']).size().reset_index(name='筆數')

    # 剔除筆數少於 5 的航班
    valid_flights = flight_counts[flight_counts['筆數'] >= 5]
    merged_data = pd.merge(merged_data, valid_flights[['出發日期', '航班代碼']], on=['出發日期', '航班代碼'], how='inner')

    # 按條件分組（出發日期、航班代碼）
    grouped = merged_data.groupby(['出發日期', '航班代碼'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median'),
        價格變異=('價格', 'var')  # 新增計算價格變異
    )
    grouped['平均價格'] = grouped['平均價格'].round().astype(int)
    grouped['價格變異'] = grouped['價格變異'].fillna(0).round(2)  # 填充 NaN 並保留到小數點後兩位

    # 計算最低價格的剩餘天數
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼'])['價格'].idxmin(),
        ['出發日期', '航班代碼', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)

    # 合併分組後的統計數據
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼'], how='inner')

    # 合併筆數資訊
    final_data = pd.merge(final_data, valid_flights, on=['出發日期', '航班代碼'], how='inner')

    # 去除重複行，只保留同一條航班的最後一筆資料
    final_data = final_data.sort_values(by=['出發日期', '航班代碼', '筆數']).drop_duplicates(
        subset=['出發日期', '航班代碼'], keep='last'
    )

    # 排序結果
    final_data = final_data.sort_values(by=['出發日期', '航班代碼']).reset_index(drop=True)

    # 儲存結果
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，結果已剔除少於 5 筆的航班，並儲存到 {output_path}")

# 主程式執行流程
base_path = '/Users/yuchingchen/Documents/專題/data'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/tokyo.csv'

# 執行合併與處理
process_and_merge_files('1021', '1127', base_path, output_path)

處理完成，結果已剔除少於 5 筆的航班，並儲存到 /Users/yuchingchen/Documents/專題/cleaned_data/tokyo.csv


資料處理-SYDNEY（按日期跟航班代碼排序）

In [15]:
import os
import pandas as pd
import re
from tabulate import tabulate  

def process_and_merge_files(start_date, end_date, base_path, output_path):
    # 建立日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2024-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/sydney_{date}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if not df.empty:
                df = df.dropna(how="all")
                
                # 格式化出發日期並去除時間
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                
                # 新增星期欄位
                df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                
                # 移除價格中的符號並轉換為數字
                df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]  # 過濾掉不符合數字格式的行
                df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                
                # 計算 day left
                file_date = pd.to_datetime(f"2024-{date[:2]}-{date[2:]}")
                df['day left'] = (pd.to_datetime(df['出發日期']) - file_date).dt.days
                
                # 定義解析主航段邏輯的函數
                def parse_duration(duration_str):
                    """解析飛行時間字串，轉換為總分鐘數"""
                    hours, minutes = 0, 0
                    if "小時" in duration_str:
                        hours = int(duration_str.split("小時")[0].strip())
                        duration_str = duration_str.split("小時")[1]
                    if "分鐘" in duration_str:
                        minutes = int(duration_str.split("分鐘")[0].strip())
                    return hours * 60 + minutes
                
                def process_main_segment(row, column_name):
                    """處理航空公司或艙等的主要段落"""
                    value = row[column_name]
                    if not isinstance(value, str):
                        return value  # 如果不是字串類型，直接返回原值
                    
                    # 特殊處理：針對「亞洲航空 X 亞洲航空 X」的情況
                    if value == "亞洲航空 X 亞洲航空 X":
                        return "亞洲航空 X"  # 直接返回處理後的結果
                    
                    # 若前後相同，直接返回其中一個
                    value = " ".join(value.split())
                    unique_segments = set(value.split(" "))
                    if len(unique_segments) == 1:  # 所有部分相同
                        return value.split(" ")[0]
                    
                    # 正常處理兩段邏輯
                    segments = value.split(" ")
                    if len(segments) == 2:
                        try:
                            first_duration = parse_duration(row["第一段飛行時間"])
                            second_duration = parse_duration(row["第二段飛行時間"])
                            return segments[0] if first_duration >= second_duration else segments[1]
                        except Exception:
                            return None
                    return value
                    
                def process_second_segment(row, column_name):
                    value = row[column_name]
                    # 確保值為字串，否則返回原始值
                    if not isinstance(value, str):
                        return value  # 如果不是字串類型，直接返回原值
                    # 使用正則表達式來精確分段
                    segments = re.findall(r'[^\s]+\s[^\s]+', value)
                    if len(segments) != 2:
                        return value  # 如果無法分辨兩段，保留原始值
                    try:
                        durations = [parse_duration(row["第一段飛行時間"]), parse_duration(row["第二段飛行時間"])]
                        return segments[0] if durations[0] >= durations[1] else segments[1]
                    except Exception:
                        return None  # 發生例外時，返回 None

                # 按邏輯處理航空公司、機型、航班代碼、艙等
                df["航空公司（主航段）"] = df.apply(lambda row: process_main_segment(row, "航空公司"), axis=1)
                df["機型（主航段）"] = df.apply(lambda row: process_second_segment(row, "機型"), axis=1)
                df["航班代碼（主航段）"] = df.apply(lambda row: process_second_segment(row, "航班代碼"), axis=1)
                df["艙等（主航段）"] = df.apply(lambda row: process_main_segment(row, "艙等"), axis=1)
                
                # 新增航空公司組合欄位
                def airline_combination(row):
                    """判斷前後航空公司是否相同"""
                    if not isinstance(row["航空公司"], str):
                        return 0
                    segments = row["航空公司"].split(" ")
                    if len(segments) == 2:
                        return 1 if segments[0] == segments[1] else 0
                    return 0  # 無法判斷時設為 0
                
                df["航空公司組合"] = df.apply(airline_combination, axis=1)
                
                # 調整欄位順序，將「航空公司（主航段）」移至「航空公司」後
                cols = df.columns.tolist()
                for col in ["航空公司（主航段）", "機型（主航段）", "航班代碼（主航段）", "艙等（主航段）"]:
                    cols.insert(cols.index(col.replace("（主航段）", "")) + 1, cols.pop(cols.index(col)))
                df = df[cols]
                
                # 調整欄位順序，將「星期」移到「出發時間」後面
                cols = df.columns.tolist()
                cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('星期')))
                df = df[cols]
                
                # 調整欄位順序，將「航空公司組合」移到「航空公司（主航段）」後
                cols = df.columns.tolist()
                cols.insert(cols.index("航空公司（主航段）") + 1, cols.pop(cols.index("航空公司組合")))
                df = df[cols]
                
                data_frames.append(df)

    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    if merged_data.empty:
        print("沒有合併到任何資料。")
        return
    
    # 刪除艙等（主航段）不等於經濟艙的數據
    merged_data = merged_data[merged_data['艙等（主航段）'] == '經濟艙']

    # 計算每個航班的筆數
    flight_counts = merged_data.groupby(['出發日期', '航班代碼']).size().reset_index(name='筆數')

    # 剔除筆數小於 10 的航班
    valid_flights = flight_counts[flight_counts['筆數'] >= 10]
    merged_data = pd.merge(merged_data, valid_flights[['出發日期', '航班代碼']], on=['出發日期', '航班代碼'], how='inner')

    # 按條件分組（出發日期、航班代碼）
    grouped = merged_data.groupby(['出發日期', '航班代碼'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median'),
        價格變異=('價格', 'var')  # 新增計算價格變異
    )
    grouped['平均價格'] = grouped['平均價格'].round().astype(int)
    grouped['價格變異'] = grouped['價格變異'].fillna(0).round(2)  # 填充 NaN 並保留到小數點後兩位

    # 計算最低價格的剩餘天數
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼'])['價格'].idxmin(),
        ['出發日期', '航班代碼', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)

    # 合併分組後的統計數據
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼'], how='inner')

    # 合併筆數資訊
    final_data = pd.merge(final_data, valid_flights, on=['出發日期', '航班代碼'], how='inner')

    # 去除重複行，只保留同一條航班的最後一筆資料
    final_data = final_data.sort_values(by=['出發日期', '航班代碼', '筆數']).drop_duplicates(
        subset=['出發日期', '航班代碼'], keep='last'
    )
    
    # 排序結果
    final_data = final_data.sort_values(by=['出發日期', '航班代碼（主航段）']).reset_index(drop=True)
    
    # 儲存結果
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，結果已剔除少於 10 筆的航班，並儲存到 {output_path}")

# 主程式執行流程
base_path = '/Users/yuchingchen/Documents/專題/data'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/sydney.csv'

# 執行合併與處理
process_and_merge_files('1021', '1127', base_path, output_path)

處理完成，結果已剔除少於 10 筆的航班，並儲存到 /Users/yuchingchen/Documents/專題/cleaned_data/sydney.csv


資料處理-SYDNEY-BUSINESS（按日期跟航班代碼排序）

In [5]:
import os
import pandas as pd
import re

def process_and_merge_files(start_date, end_date, base_path, output_path):
    # 建立日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2024-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/sydney_business_{date}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if not df.empty:
                df = df.dropna(how="all")
                
                # 格式化出發日期並去除時間
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                
                # 新增星期欄位
                df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                
                # 移除價格中的符號並轉換為數字
                df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]  # 過濾掉不符合數字格式的行
                df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                
                # 計算 day left
                file_date = pd.to_datetime(f"2024-{date[:2]}-{date[2:]}")
                df['day left'] = (pd.to_datetime(df['出發日期']) - file_date).dt.days
                
                # 定義解析主航段邏輯的函數
                def parse_duration(duration_str):
                    """解析飛行時間字串，轉換為總分鐘數"""
                    hours, minutes = 0, 0
                    if "小時" in duration_str:
                        hours = int(duration_str.split("小時")[0].strip())
                        duration_str = duration_str.split("小時")[1]
                    if "分鐘" in duration_str:
                        minutes = int(duration_str.split("分鐘")[0].strip())
                    return hours * 60 + minutes
                
                def process_main_segment(row, column_name):
                    """處理航空公司或艙等的主要段落"""
                    value = row[column_name]
                    if not isinstance(value, str):
                        return value  # 如果不是字串類型，直接返回原值
                    
                    # 特殊處理：針對「亞洲航空 X 亞洲航空 X」的情況
                    if value == "亞洲航空 X 亞洲航空 X":
                        return "亞洲航空 X"  # 直接返回處理後的結果
                    
                    # 若前後相同，直接返回其中一個
                    value = " ".join(value.split())
                    unique_segments = set(value.split(" "))
                    if len(unique_segments) == 1:  # 所有部分相同
                        return value.split(" ")[0]
                    
                    # 正常處理兩段邏輯
                    segments = value.split(" ")
                    if len(segments) == 2:
                        try:
                            first_duration = parse_duration(row["第一段飛行時間"])
                            second_duration = parse_duration(row["第二段飛行時間"])
                            return segments[0] if first_duration >= second_duration else segments[1]
                        except Exception:
                            return None
                    return value
                    
                def process_second_segment(row, column_name):
                    value = row[column_name]
                    # 確保值為字串，否則返回原始值
                    if not isinstance(value, str):
                        return value  # 如果不是字串類型，直接返回原值
                    # 使用正則表達式來精確分段
                    segments = re.findall(r'[^\s]+\s[^\s]+', value)
                    if len(segments) != 2:
                        return value  # 如果無法分辨兩段，保留原始值
                    try:
                        durations = [parse_duration(row["第一段飛行時間"]), parse_duration(row["第二段飛行時間"])]
                        return segments[0] if durations[0] >= durations[1] else segments[1]
                    except Exception:
                        return None  # 發生例外時，返回 None

                # 按邏輯處理航空公司、機型、航班代碼、艙等
                df["航空公司（主航段）"] = df.apply(lambda row: process_main_segment(row, "航空公司"), axis=1)
                df["機型（主航段）"] = df.apply(lambda row: process_second_segment(row, "機型"), axis=1)
                df["航班代碼（主航段）"] = df.apply(lambda row: process_second_segment(row, "航班代碼"), axis=1)
                df["艙等（主航段）"] = df.apply(lambda row: process_main_segment(row, "艙等"), axis=1)
                
                # 調整欄位順序，將「航空公司（主航段）」移至「航空公司」後
                cols = df.columns.tolist()
                for col in ["航空公司（主航段）", "機型（主航段）", "航班代碼（主航段）", "艙等（主航段）"]:
                    cols.insert(cols.index(col.replace("（主航段）", "")) + 1, cols.pop(cols.index(col)))
                df = df[cols]
                
                # 調整欄位順序，將「星期」移到「出發時間」後面
                cols = df.columns.tolist()
                cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('星期')))
                df = df[cols]
                
                data_frames.append(df)

    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    if merged_data.empty:
        print("沒有合併到任何資料。")
        return

    # 計算每個航班的筆數
    flight_counts = merged_data.groupby(['出發日期', '航班代碼']).size().reset_index(name='筆數')

    # 剔除筆數小於 10 的航班
    valid_flights = flight_counts[flight_counts['筆數'] >= 10]
    merged_data = pd.merge(merged_data, valid_flights[['出發日期', '航班代碼']], on=['出發日期', '航班代碼'], how='inner')

    # 按條件分組（出發日期、航班代碼）
    grouped = merged_data.groupby(['出發日期', '航班代碼'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median')
    )
    grouped['平均價格'] = grouped['平均價格'].round().astype(int)

    # 計算最低價格的剩餘天數
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼'])['價格'].idxmin(),
        ['出發日期', '航班代碼', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)

    # 合併分組後的統計數據
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼'], how='inner')

    # 合併筆數資訊
    final_data = pd.merge(final_data, valid_flights, on=['出發日期', '航班代碼'], how='inner')

    # 去除重複行，只保留同一條航班的最後一筆資料
    final_data = final_data.sort_values(by=['出發日期', '航班代碼', '筆數']).drop_duplicates(
        subset=['出發日期', '航班代碼'], keep='last'
    )
    
    # 排序結果
    final_data = final_data.sort_values(by=['出發日期', '航班代碼']).reset_index(drop=True)

    # 儲存結果
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，結果已剔除少於 10 筆的航班，並儲存到 {output_path}")


# 主程式執行流程
base_path = '/Users/yuchingchen/Documents/專題/data'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/sydney_business.csv'

# 執行合併與處理
process_and_merge_files('1021', '1126', base_path, output_path)

處理完成，結果已剔除少於 5 筆的航班，並儲存到 /Users/yuchingchen/Documents/專題/cleaned_data/sydney_business.csv
